<a href="https://colab.research.google.com/github/MD2001/Pytorch_Bignnier/blob/main/01_pytoirch_WorkFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PyTorch Workflow

Let's explore a an example PyTorch end-to-end workflow.

Resources:
* Ground truth notebook - https://github.com/mrdbourke/pytorch-deep-learning/blob/main/01_pytorch_workflow.ipynb
* Book version of notebook - https://www.learnpytorch.io/01_pytorch_workflow/
* Ask a question- https://github.com/mrdbourke/pytorch-deep-learning/discussions

In [9]:
import torch
from torch import nn ## has all bulding block for nueral network
import matplotlib.pyplot as plt

# Data (preparing and loading)

Data can be almost anything... in machine learning.
* Excel speadsheet
* Images of any kind
* Videos (YouTube has lots of data...)
* Audio like songs or podcasts
* DNA
* Text
Machine learning is a game of two parts:
1. Get data into a numerical representation.
2. Build a model to learn patterns in that numerical representation.

To showcase this, let's create some *known* data using the linear regression formula.

We'll use a linear regression formula to make a straight line with *known* **parameters**.|

In [10]:
#parameters
wight=0.7
bias=0.3
#create tensor
start=0
end=1
step=0.02

X=torch.arange(start,end,step).unsqueeze(dim=1)

Y=wight*X+bias

print(X[:10],Y[:10])
print(len(X),len(Y))



tensor([[0.0000],
        [0.0200],
        [0.0400],
        [0.0600],
        [0.0800],
        [0.1000],
        [0.1200],
        [0.1400],
        [0.1600],
        [0.1800]]) tensor([[0.3000],
        [0.3140],
        [0.3280],
        [0.3420],
        [0.3560],
        [0.3700],
        [0.3840],
        [0.3980],
        [0.4120],
        [0.4260]])
50 50


### Createing the training set and test set for our data set

In [14]:
#should but spliting ritoe about 80%:20%
traing_split=int(0.8*len(X))
x_train,y_train=X[:traing_split],Y[:traing_split]
x_test,y_test=X[traing_split:],Y[traing_split:]
len(x_train),len(x_test) ,len(y_train),len(y_test)

(40, 10, 40, 10)